In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np

In [74]:
df1 = pd.read_csv(r"../data/winequality-red.csv",decimal=",",delimiter=";")
df2 = pd.read_csv(r"../data/winequality-white.csv",decimal=",",delimiter=";")
df1.insert (loc= len(df1.columns) , column='color', value=6)
df2.insert (loc= len(df2.columns) , column='color', value=7)
data = pd.concat([df1, df2], axis = 0,ignore_index = True)
data['fixed acidity'] = data['fixed acidity'].astype(float)
data['volatile acidity'] = data['volatile acidity'].astype(float)
data['citric acid'] = data['citric acid'].astype(float)
data['residual sugar'] = data['residual sugar'].astype(float)
data['chlorides'] = data['chlorides'].astype(float)
data['free sulfur dioxide'] = data['free sulfur dioxide'].astype(float)
data['total sulfur dioxide'] = data['total sulfur dioxide'].astype(float)
data['density'] = data['density'].astype(float)
data['pH'] = data['pH'].astype(float)
data['sulphates'] = data['sulphates'].astype(float)
data['alcohol'] = data['alcohol'].astype(float)
data.drop_duplicates(inplace=True)
data.duplicated().sum()
outlier=x = data.drop(["color", "quality"], axis=1)
Q1 = outlier.quantile(0.25)
Q3 = outlier.quantile(0.75)
IQR = Q3 - Q1
data_f = outlier[~((outlier < (Q1 - 1.5 * IQR)) | (outlier > (Q3 + 1.5 * IQR))).any(axis=1)]
index_list = list(data_f.index.values)
data_f = data[data.index.isin(index_list)]
datareg=data_f

y_reg = data["quality"]
x_reg = data.drop("quality", axis=1)
x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(x_reg, y_reg, test_size=0.2)

In [75]:
data=pd.read_csv("../data/weatherAUS.csv")
data.dropna(inplace=True,ignore_index=True)
f = lambda x : str(x)[5:7]
data['Date'] = data['Date'].transform(f)
data['Date'] = data['Date'].astype(int)
f = lambda x : 0 if (x == "No") else 1
data['RainToday'] = data['RainToday'].transform(f)
data['RainToday'] = data['RainToday'].astype(int)

data['RainTomorrow'] = data['RainTomorrow'].transform(f)
data['RainTomorrow'] = data['RainTomorrow'].astype(int)
categorical_features = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
categorical_encoder = OneHotEncoder(sparse_output=False)
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[
     ('cat', categorical_encoder, categorical_features)
     ])
# Получите закодированные функции в виде DataFrame.
ct.set_output(transform='pandas')
encoded_features = ct.fit_transform(data)
encoded_features.head()
data=data.drop(['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'],axis=1)
df = pd.concat([
    data,
    encoded_features
], axis=1)
df_major_0 = df[df["RainTomorrow"] == 0]
df_minor_1 = df[df["RainTomorrow"] == 1]

df_major_0_undersampled = df_major_0.sample(len(df_minor_1))

dataclass = pd.concat([df_major_0_undersampled, df_minor_1], axis=0)

x_class=dataclass.drop(['RainTomorrow'],axis=1)
y_class=dataclass['RainTomorrow']

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(x_class, y_class, test_size=0.33)

In [76]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(12,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)
model_regression.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 dense_98 (Dense)            (None, 64)                832       
                                                                 
 dense_99 (Dense)            (None, 32)                2080      
                                                                 
 dropout_30 (Dropout)        (None, 32)                0         
                                                                 
 dense_100 (Dense)           (None, 16)                528       
                                                                 
 dropout_31 (Dropout)        (None, 16)                0         
                                                                 
 dense_101 (Dense)           (None, 1)                 17        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [77]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [78]:
model_regression.fit(x_train_r, y_train_r, epochs=50)

Epoch 1/50


133/133 [==============================] - 1s 1ms/step - loss: 8.3167
Epoch 2/50
133/133 [==============================] - 0s 1ms/step - loss: 2.2387
Epoch 3/50
133/133 [==============================] - 0s 1ms/step - loss: 1.9172
Epoch 4/50
133/133 [==============================] - 0s 1ms/step - loss: 1.7997
Epoch 5/50
133/133 [==============================] - 0s 1ms/step - loss: 1.6955
Epoch 6/50
133/133 [==============================] - 0s 1ms/step - loss: 1.5082
Epoch 7/50
133/133 [==============================] - 0s 1ms/step - loss: 1.4175
Epoch 8/50
133/133 [==============================] - 0s 1ms/step - loss: 1.3786
Epoch 9/50
133/133 [==============================] - 0s 1ms/step - loss: 1.2680
Epoch 10/50
133/133 [==============================] - 0s 1ms/step - loss: 1.1897
Epoch 11/50
133/133 [==============================] - 0s 1ms/step - loss: 1.1983
Epoch 12/50
133/133 [==============================] - 0s 1ms/step - loss: 1.1024
Epoch 13/50
133/133 [===============

In [79]:
print(mean_absolute_error(y_test_r, model_regression.predict(x_test_r)))
print(mean_squared_error(y_test_r, model_regression.predict(x_test_r)))

34/34 [==============================] - 0s 849us/step
0.5784535645542288
34/34 [==============================] - 0s 929us/step
0.5537894641894647


In [104]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(92,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
# verbose=None - не будет логов
model_classification_1.fit(x_train_c, y_train_c, epochs=25, verbose=None)

y_pred = np.around(model_classification_1.predict(x_test_c, verbose=None))

print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      4099
           1       0.80      0.79      0.80      4103

    accuracy                           0.80      8202
   macro avg       0.80      0.80      0.80      8202
weighted avg       0.80      0.80      0.80      8202

[[3298  801]
 [ 849 3254]]


In [82]:
w0 = 1 / y_train_c[y_train_c==0].shape[0]
w1 = 1 / y_train_c[y_train_c==1].shape[0]

In [105]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_class.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification_1.fit(x_train_c, y_train_c, epochs=25, verbose=None, class_weight={0: w0, 1: w1})
model_classification_1.predict(x_test_c, verbose=None)

y_pred = np.around(model_classification_1.predict(x_test_c, verbose=None))
print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80      4099
           1       0.79      0.80      0.80      4103

    accuracy                           0.80      8202
   macro avg       0.80      0.80      0.80      8202
weighted avg       0.80      0.80      0.80      8202

[[3248  851]
 [ 817 3286]]


In [110]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_dim = x_class.shape[1]),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(x_train_c, y_train_c, epochs=25, verbose=None)


In [111]:
y_pred = [np.argmax(pred) for pred in model_classification_2.predict(x_test_c, verbose=None)]
print(classification_report(y_test_c, y_pred))
print(confusion_matrix(y_test_c, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.72      0.78      4099
           1       0.76      0.87      0.81      4103

    accuracy                           0.79      8202
   macro avg       0.80      0.79      0.79      8202
weighted avg       0.80      0.79      0.79      8202

[[2948 1151]
 [ 540 3563]]
